In [ ]:
# Initiate DB
from src.db_models.base_model import db
from src.db_models.sample import Sample
from src.db_models.ml_model import MlModel
from src.db_models.prediction import Prediction
from src.db_models.test_sample import TestSample

# Connect to DB
db.connect()
# Migrate tables
db.create_tables([MlModel, TestSample, Sample, Prediction])


In [ ]:
# Get data
import pandas as pd

data = pd.read_csv("../data/Boiler_emulator_dataset_new.csv")

predictions = data["Class"]
features = data.drop(columns=['Condition', 'Class'])

In [ ]:
# Create Primary model
model_file = "models/model_v1.pb"

MlModel.update(is_active=False).execute()
model_record = MlModel.create(model_path=model_file, is_active=True)

In [ ]:
# Fill DB with data

from sklearn.model_selection import train_test_split

X_train_full , X_test , y_train_full , y_test = train_test_split(features, predictions, random_state=42)

# Insert to DB
for index, row in X_test.iterrows():
    sample = TestSample.create(
        fuel_mdot = row["Fuel_Mdot"],
        tair = row["Tair"],
        treturn = row["Treturn"],
        tsupply = row["Tsupply"],
        water_mdot = row["Water_Mdot"])
        
X_test

In [ ]:
db.close()